In [7]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters = 1000
# eval_interval = 2500
learning_rate = 3e-4
eval_iters = 250

cpu


In [8]:
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

#this will create a list of all the characters in the text file
#the set function will remove all the duplicates
#the sorted function will sort the characters in the list
   #use a tokenizer to convert each element into  an integer

    #

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


In [9]:
string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([80, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,  1, 47,
        33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26, 49,  0,
         0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,  0,  0,
         1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1, 47, 33,
        50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1, 36, 25,
        38, 28,  1, 39, 30,  1, 39, 50,  9,  1])


In [10]:
n = int(0.8 * len(data))
train_data = data[:n]
val_data = data[n:] 

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - (block_size), (batch_size,))
    print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs:')
# print(x.shape)
print(x)
print('targets:') 
print(y)


tensor([145796, 131139, 128472, 114641])
inputs:
tensor([[73,  1, 57, 62, 72, 73, 54, 67],
        [ 1, 73, 61, 58,  1, 31, 54, 71],
        [57, 75, 54, 67, 56, 58, 57,  1],
        [11,  1, 33, 73,  1, 62, 72,  1]])
targets:
tensor([[ 1, 57, 62, 72, 73, 54, 67, 56],
        [73, 61, 58,  1, 31, 54, 71, 60],
        [75, 54, 67, 56, 58, 57,  1, 74],
        [ 1, 33, 73,  1, 62, 72,  1, 54]])


In [11]:
## Biogram data model is a model that predicts the next character given the previous character  
## bio means two so we're going to use two characters to predict the next character

x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(y)
    print('when input is', context, 'target is', target)


tensor([28, 39, 42, 39, 44, 32, 49,  1])
when input is tensor([80]) target is tensor(28)
tensor([28, 39, 42, 39, 44, 32, 49,  1])
when input is tensor([80, 28]) target is tensor(39)
tensor([28, 39, 42, 39, 44, 32, 49,  1])
when input is tensor([80, 28, 39]) target is tensor(42)
tensor([28, 39, 42, 39, 44, 32, 49,  1])
when input is tensor([80, 28, 39, 42]) target is tensor(39)
tensor([28, 39, 42, 39, 44, 32, 49,  1])
when input is tensor([80, 28, 39, 42, 39]) target is tensor(44)
tensor([28, 39, 42, 39, 44, 32, 49,  1])
when input is tensor([80, 28, 39, 42, 39, 44]) target is tensor(32)
tensor([28, 39, 42, 39, 44, 32, 49,  1])
when input is tensor([80, 28, 39, 42, 39, 44, 32]) target is tensor(49)
tensor([28, 39, 42, 39, 44, 32, 49,  1])
when input is tensor([80, 28, 39, 42, 39, 44, 32, 49]) target is tensor(1)


In [ ]:
class BigGramLanguageModel(nn.Module):
    